In [3]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

from sklearn.preprocessing import StandardScaler
from category_encoders import OneHotEncoder

import joblib
from skimpy import skim

def prepare_data(data):
    df_raw_new = pd.read_csv(data)
    
    if 'Loan_ID' in df_raw_new.columns:
        df_raw_new.drop(columns='Loan_ID', inplace=True)

    df_raw_new['Credit_History'] = df_raw_new['Credit_History'].astype(object)
    missing_values = (df_raw_new.isnull().sum() / len(df_raw_new) * 100).round(0).astype(int)

    df_raw_new['LoanAmount'].fillna(df_raw_new['LoanAmount'].mean(), inplace=True)
    df_raw_new['Loan_Amount_Term'].fillna(df_raw_new['Loan_Amount_Term'].mode()[0], inplace=True)
    df_raw_new['Credit_History'].fillna(df_raw_new['Credit_History'].mode()[0], inplace=True)
    
    df_raw_new['Gender'].fillna(df_raw_new['Gender'].mode()[0], inplace=True)
    df_raw_new['Married'].fillna(df_raw_new['Married'].mode()[0], inplace=True)
    df_raw_new['Dependents'].fillna(df_raw_new['Dependents'].mode()[0], inplace=True)
    df_raw_new['Self_Employed'].fillna(df_raw_new['Self_Employed'].mode()[0], inplace=True)

    min_mask = lambda col, val: df_raw_new[col] < val
    income_mask = min_mask('ApplicantIncome', 10000)
    coapplicant_income_mask = min_mask('CoapplicantIncome', 5701)
    loan_amount_mask = min_mask('LoanAmount', 260)

    df_raw_new1 = df_raw_new[income_mask & coapplicant_income_mask & loan_amount_mask]

    if 'Loan_Status' in df_raw_new1.columns:
        df_raw_new1['Loan_Status'] = df_raw_new1['Loan_Status'].apply(lambda s: 1 if s == 'Y' else 0)
    
    df_raw_new1['Total_Income'] = df_raw_new1['ApplicantIncome'] + df_raw_new1['CoapplicantIncome']
    df_raw_new1['Loan_Term_Category'] = pd.cut(df_raw_new1['Loan_Amount_Term'], bins=[0, 180, 360, 600], labels=['Short-term', 'Medium-term', 'Long-term'])
    df_raw_new1['Income_Stability'] = df_raw_new1[['ApplicantIncome', 'CoapplicantIncome']].std(axis=1)
    df_raw_new1['Loan_to_Income_ratio'] = df_raw_new1['LoanAmount'] / (df_raw_new1['ApplicantIncome'] + df_raw_new1['CoapplicantIncome'])
    
    df_raw_new1['Loan_Term_Category'] = df_raw_new1['Loan_Term_Category'].astype(object)
        
    ohe = OneHotEncoder(
        use_cat_names=True, 
        cols=['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area', 'Loan_Term_Category']
    )

    encoded_df = ohe.fit_transform(df_raw_new1)

    return encoded_df


def make_prediction(data):
    prediction_data = data
    model = joblib.load('../artifacts/model2.pkl')
    
    scaler = StandardScaler()
    scaler.fit(prediction_data) 

    scaled_input = scaler.transform(prediction_data)

    predictions = model.predict(scaled_input)
    
    last_prediction = 'Yes' if predictions[-1] > 0.5 else 'No'
    
    return last_prediction






app = dash.Dash(__name__)


input_style = {'width': '80%', 'margin': '10px'}
label_style = {'font-weight': 'bold', 'margin-right': '10px', 'display': 'inline-block', 'width': '150px'}


centered_box_style = {
    'width': '50%',
    'margin': 'auto',
    'padding': '50px',
    'border': '1px solid #ddd',
    'border-radius': '10px',
    'background-color': '#ffffff' 
}


full_page_style = {
    'background-color': '#adc5cf',  
    'height': '100vh',             
    'display': 'flex',              
    'justify-content': 'center',   
    'align-items': 'center'         
}


app.layout = html.Div(style=full_page_style, children=[
    html.Div(style=centered_box_style, children=[
        html.H1("Loan Application", style={'text-align': 'center', 'margin-bottom': '30px'}),

    html.Div([
        html.Label("Gender:", style=label_style),
        dcc.Dropdown(
            id='gender-dropdown',
            options=[
                {'label': 'Male', 'value': 'Male'},
                {'label': 'Female', 'value': 'Female'}
            ],
            placeholder="Select gender",
            style=input_style
        )
    ]),

    html.Div([
        html.Label("Married:", style=label_style),
        dcc.Dropdown(
            id='married-dropdown',
            options=[
                {'label': 'Yes', 'value': 'Yes'},
                {'label': 'No', 'value': 'No'}
            ],
            placeholder="Marital status",
            style=input_style
        )
    ]),

    html.Div([
        html.Label("Dependents:", style={'font-weight': 'bold', 'margin-right': '10px'}),
        dcc.Dropdown(
            id='dependents-dropdown',
            options=[
                {'label': '0', 'value': 0},
                {'label': '1', 'value': 1},
                {'label': '2', 'value': 2},
                {'label': '3+', 'value': '3+'}
            ],
            placeholder="Number of dependents",
            style={'width': '80%', 'margin': '10px'}
        )
    ]),

    html.Div([
        html.Label("Education:", style=label_style),
        dcc.Dropdown(
            id='education-dropdown',
            options=[
                {'label': 'Graduate', 'value': 'Graduate'},
                {'label': 'Not Graduate', 'value': 'Not Graduate'}
            ],
            placeholder="Education level",
            style=input_style
        )
    ]),

    html.Div([
        html.Label("Self Employed:", style=label_style),
        dcc.Dropdown(
            id='self-employed-dropdown',
            options=[
                {'label': 'Yes', 'value': 'Yes'},
                {'label': 'No', 'value': 'No'}
            ],
            placeholder="Employment status",
            style=input_style
        )
    ]),
    
        html.Div([
        html.Label("ApplicantIncome:", style=label_style),
        dcc.Input(
            id='applicant-income-input',
            type='number',
            placeholder="Enter your income",
            style=input_style
        )
    ]),
    
        html.Div([
        html.Label("Coapplicant Income:", style=label_style),
        dcc.Input(
            id='coapplicant-income-input',
            type='number',
            placeholder="Enter your coapplicants income",
            style=input_style
        )
    ]),
        

    html.Div([
        html.Label("Loan Amount (in Rands):", style=label_style),
        dcc.Input(
            id='loan-amount-input',
            type='number',
            placeholder="Enter loan amount",
            style=input_style
        )
    ]),

    html.Div([
            html.Label("Loan Term (Months):", style=label_style),
            dcc.Dropdown(
                id='loan-term-dropdown',
                options=[{'label': f'{months} months', 'value': months} for months in range(12, 481, 12)],
                placeholder="Loan term in months",
                style=input_style
            )
        ]),

    html.Div([
        html.Label("Credit History:", style=label_style),
        dcc.Dropdown(
            id='credit-history-dropdown',
            options=[
                {'label': 'Yes', 'value': 'Yes'},
                {'label': 'No', 'value': 'No'}
            ],
            placeholder="Credit history",
            style=input_style
        )
    ]),

    html.Div([
        html.Label("Property Area:", style=label_style),
        dcc.Dropdown(
            id='property-area-dropdown',
            options=[
                {'label': 'Urban', 'value': 'Urban'},
                {'label': 'Rural', 'value': 'Rural'},
                {'label': 'Semiurban', 'value': 'Semiurban'}
            ],
            placeholder="Select property area",
            style=input_style
        )
    ]),

    html.Button('Submit', id='submit-button', n_clicks=0, style={'margin-top': '30px'}),

    html.Div(id='output-message', style={'margin-top': '20px', 'font-size': '18px', 'font-weight': 'bold'})
])
])

def validate_input(value):
    return value is not None and str(value).strip() != ""

def writeToFile(gender, married, dependents, education, self_employed, applicant_income, coapplicant_income,
                loan_amount, loan_term, credit_history, property_area):
    if not all(validate_input(val) for val in [gender, married, dependents, education, self_employed,
                                            applicant_income, coapplicant_income, loan_amount,
                                            loan_term, credit_history, property_area]):
        print("Error: One or more required fields are missing or invalid.")
        return
    
    credit_history_value = 1 if credit_history == 'Yes' else 0
    
    data = {
        'Loan_ID': [1],
        'Gender': [gender],
        'Married': [married],
        'Dependents': [dependents],
        'Education': [education],
        'Self_Employed': [self_employed],
        'ApplicantIncome': [applicant_income],
        'CoapplicantIncome': [coapplicant_income],
        'LoanAmount': [loan_amount],
        'Loan_Amount_Term': [loan_term],
        'Credit_History': [credit_history_value],
        'Property_Area': [property_area]
    }
    
    df = pd.DataFrame(data)
    
    csv_file_path = '../data/loan_application.csv'
    
    if df.notna().all().all():
        with open(csv_file_path, 'a', newline='') as f:
            if f.tell() == 0:
                df.to_csv(f, index=False)
            else:
                df.to_csv(f, header=False, index=False)
        print(f"Loan details appended successfully to '{csv_file_path}'.")
    else:
        print("Error: One or more required fields are missing or invalid.")



@app.callback(
    Output('output-message', 'children'),
    [Input('submit-button', 'n_clicks')],
    [State('gender-dropdown', 'value'),
    State('married-dropdown', 'value'),
    State('dependents-dropdown', 'value'),
    State('education-dropdown', 'value'),
    State('self-employed-dropdown', 'value'),
    State('applicant-income-input', 'value'),
    State('coapplicant-income-input', 'value'),
    State('loan-amount-input', 'value'),
    State('loan-term-dropdown', 'value'),
    State('credit-history-dropdown', 'value'),
    State('property-area-dropdown', 'value')]
)
def update_output(n_clicks, gender, married, dependents, education, self_employed, applicant_income,
                coapplicant_income, loan_amount, loan_term, credit_history, property_area):
    if n_clicks > 0:
        writeToFile(gender, married, dependents, education, self_employed, applicant_income, coapplicant_income,
                    loan_amount, loan_term, credit_history, property_area)
        df = prepare_data("../data/loan_application.csv")

        return f"Prediction: {make_prediction(df)}"
    else:
        return ''

if __name__ == '__main__':
    app.run_server(debug=True, port=1001)
